# 특정 사이트 크롤링

In [1]:
from IPython.display import Image
from selenium import webdriver       # 1. Java, 2. Python
from selenium.webdriver.common.keys import Keys
from time import sleep
import requests
from bs4 import BeautifulSoup 
import re
import shutil
import os
from selenium.webdriver.support.ui import Select #select -> option 구문으로 작업 할 수있는 클래스
import pandas as pd

## 함수

In [2]:
# Xpath로 접근 후 클릭(버튼 같은 경우)
def click_xp(a):
    # element = driver.find_element_by_xpath('xpath')
    #driver.execute_script("arguments[0].click();", element)
    driver.find_element_by_xpath(a).click()

In [3]:
# 아이디로 접근 후 클릭
def click_id(a):
    element=driver.find_element_by_id(a)
    element.click()

In [4]:
# 로그인 ~~~! 
def Login(id, password):
    #id 입력과정
    elem_id = driver.find_element_by_id("email_view")# 아이디 입력칸의 ID 넣기
    elem_id.send_keys(id)
    driver.implicitly_wait(5)
    
    # password 입력 과정
    elem_pw = driver.find_element_by_id("password_view")# password 입력칸의 ID 넣기
    elem_pw.send_keys(password)
    driver.implicitly_wait(5)
    
    # 로그인 버튼 클릭
    driver.find_element_by_xpath("""//*[@id="content"]/div[2]/div/input""").click() # 로그인 버튼의 Xpath 넣기
    driver.implicitly_wait(5)
    sleep(2)

In [5]:
### 디렉토리 생성
def B_C_D(a):
    dir_path = "C:\\Users\\이민호\\Desktop\\crawling\\data(영어)"
    dir_name = a
   
    try:
        # 현재 경로에 동일한 폴더가 있는지 파악
        if not(os.path.isdir(a)):
            os.mkdir(dir_path + "\\" + dir_name + "\\")
    except OSError as e:
        pass  

In [6]:
### class 받아오기 ###
def cls_m(a):
    
    url = driver.page_source # 현재 페이지의 html를 가져온다
    soup = BeautifulSoup(url, 'lxml') #어떤 분석기(parser)를 사용할지 지정
    big_col = soup.select(a) #  발견한 모든 엘리먼트를 리스트 형식으로 반환
    
    #print(big_col)
    
    a = str(big_col[0])
    b = a.split('/option>')
    c = []
    cls = []
    
    for i in range(len(b)):
        match = re.findall(r'>(.+)<', b[i])
        if (match == []):
            pass
        else:
            c.append(match[0])
            out = re.findall(r'(.+)분류', c[i])
            if (out == []):
                cls.append(match[0])
            else:
                pass
    
    for i in range(len(cls)):
        out = re.search(r'(.+)/(.+)', cls[i])
        if out == None:
            pass
        else:
            d = out.group().split('/')
            f = '_or_'.join(d)
            cls[i] = f
            
        
    return(cls)

In [7]:
def check_empty(string): 
    url = driver.page_source
    soup = BeautifulSoup(url, 'lxml')
    css = soup.select(string) # 해당 부분을 선택한 후 리스트로 출력
    #print(string)
    a = str(css[0])
    out = re.findall('검색결과가 없습니다.', a)

    if (out == []):
        pass
    else:
        return 'empty'

In [8]:
### page 수 받아오기 ###
def page_num():
    
    url = driver.page_source
    soup = BeautifulSoup(url, 'lxml')
    css = soup.select("""#content > div.pageNavi > span""") # 번호부분의 selector
    #
    
    a = str(css[0])
    b = a.split('/a>')
    c = 0
    
    out = re.findall(r'2', b[0])
    if (out == []):
        c = 1
    else:
        c = len(b)

    return(c)

## Main Source

In [ ]:
driver = webdriver.Chrome('chromedriver.exe') # 크롬 드라이버 로드
driver.get('http://api.visitkorea.or.kr/main.do')# 홈페이지 접근

click_xp("""//*[@id="headerForm"]/ul[2]/li[2]/a""") #로그인을 위한 로그인 버튼 누르기
driver.implicitly_wait(5)

Login('MY ID', 'MY PW')

# 크롤링할 페이지 접근
click_xp("""//*[@id="gnb"]/nav/a/img""")
driver.implicitly_wait(5)
click_xp("""//*[@id="gnb"]/nav/ul/li[3]/ul/li[2]/a""")
driver.implicitly_wait(5)

# ratio button 클릭
click_id("rlangType_ENG")
sleep(1)
driver.implicitly_wait(1) # 브라우저 자체가 웹 요소들을 기다리도록 만들어주는 옵션

# check box 클릭
click_id("typeid_80")
driver.implicitly_wait(1)
sleep(1)

### 50개로 보기 ###        
click_xp("""//*[@id="numOfPage"]/option[5]""")
click_xp("""//*[@id="numOfPageView"]""")
sleep(1)

# 대분류 리스트 만들기
big_col = cls_m("""#selectDiv > table > tbody > tr:nth-child(2) > td > div:nth-child(1) > span:nth-child(1) > select:nth-child(1)""")

sleep(1)
driver.implicitly_wait(3)

for i in range(len(big_col)):
    num = str(i+2) 
    # //*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[1]/option[2], //*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[1]/option[3]
    xpath = """//*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[1]/option[""" + num + "]" # '' 와 "", ''''''이 사용가능
    click_xp(xpath) 
    
    #대분류 접속
    B_C_D(big_col[i]) 
    driver.implicitly_wait(5)
    sleep(1)
    
    ###중분류 받아오기###
    mid_col = cls_m("""#selectDiv > table > tbody > tr:nth-child(2) > td > div:nth-child(1) > span:nth-child(1) > select:nth-child(2)""")
    
    for l in range(len(mid_col)):
        num = str(l+2)
        xpath2 = """//*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[2]/option[""" + num + "]"
        click_xp(xpath2) 
        #중분류 접속 
        # M_C_D(big_col[i], mid_col[l])
        driver.implicitly_wait(3)
        sleep(1)
        
        ###소분류 받아오기
        small_col = cls_m("""#selectDiv > table > tbody > tr:nth-child(2) > td > div:nth-child(1) > span:nth-child(1) > select:nth-child(3)""")
        
        for m in range(len(small_col)):
            num = str(m+2)
            xpath3 = """//*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[3]/option[""" + num + "]"
            click_xp(xpath3)
            # S_C_D(big_col[i], mid_col[l], small_col[m])
            #소분류 접속
            driver.implicitly_wait(3)
            sleep(1) 
            
            click_xp("""//*[@id="search"]""")
            
            sleep(2)
            driver.implicitly_wait(3)
            #페이지가 바뀔땐 좀더 기다립시다.
            
            
            if check_empty("#listForm > ul") =='empty': # 데이터가 없는 경우
                print("[{}][{}][{}] : empty".format(big_col[i], mid_col[l],small_col[m]))
            else:
                print("[{}][{}][{}] : {}".format(big_col[i], mid_col[l],small_col[m],page_num()))
                # downloader(big_col[i], page_num())
            driver.implicitly_wait(3)
            sleep(5)
            


In [40]:
# 대분류 리스트 만들기
big_col = cls_m("""#selectDiv > table > tbody > tr:nth-child(2) > td > div:nth-child(1) > span:nth-child(1) > select:nth-child(1)""")

sleep(1)
driver.implicitly_wait(3)

for i in range(len(big_col)):
    num = str(i+2) 
    # //*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[1]/option[2], //*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[1]/option[3]
    xpath = """//*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[1]/option[""" + num + "]" # '' 와 "", ''''''이 사용가능
    click_xp(xpath) 
    
    #대분류 접속
    B_C_D(big_col[i]) 
    driver.implicitly_wait(5)
    sleep(1)
    
    ###중분류 받아오기###
    mid_col = cls_m("""#selectDiv > table > tbody > tr:nth-child(2) > td > div:nth-child(1) > span:nth-child(1) > select:nth-child(2)""")
    
    for l in range(len(mid_col)):
        num = str(l+2)
        xpath2 = """//*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[2]/option[""" + num + "]"
        click_xp(xpath2) 
        #중분류 접속 
        # M_C_D(big_col[i], mid_col[l])
        driver.implicitly_wait(3)
        sleep(1)
        
        ###소분류 받아오기
        small_col = cls_m("""#selectDiv > table > tbody > tr:nth-child(2) > td > div:nth-child(1) > span:nth-child(1) > select:nth-child(3)""")
        
        for m in range(len(small_col)):
            num = str(m+2)
            xpath3 = """//*[@id="selectDiv"]/table/tbody/tr[2]/td/div[1]/span[1]/select[3]/option[""" + num + "]"
            click_xp(xpath3)
            # S_C_D(big_col[i], mid_col[l], small_col[m])
            #소분류 접속
            driver.implicitly_wait(3)
            sleep(1) 
            
            click_xp("""//*[@id="search"]""")
            
            sleep(2)
            driver.implicitly_wait(3)
            #페이지가 바뀔땐 좀더 기다립시다.
            
            
            if check_empty("#listForm > ul") =='empty': # 데이터가 없는 경우
                print("[{}][{}][{}] : empty".format(big_col[i], mid_col[l],small_col[m]))
            else:
                print("[{}][{}][{}] : {}".format(big_col[i], mid_col[l],small_col[m],page_num()))
                # downloader(big_col[i], page_num())
            driver.implicitly_wait(3)
            sleep(5)
            


[Nature][Natural Sites][National Parks] : 2
[Nature][Natural Sites][Provincial Parks] : 2
[Nature][Natural Sites][County Parks] : 1
[Nature][Natural Sites][Mountains] : 6
[Nature][Natural Sites][Eco-Tourism Sites] : 5
[Nature][Natural Sites][Recreational Forests] : 5
[Nature][Natural Sites][Botanical Gardens] : 3
[Nature][Natural Sites][Waterfalls] : 1
[Nature][Natural Sites][Valleys] : 4
[Nature][Natural Sites][Mineral Springs] : 1
[Nature][Natural Sites][Coastal Attractions] : 2
[Nature][Natural Sites][Beaches] : 6
[Nature][Natural Sites][Islands] : 4


KeyboardInterrupt: 